In [ ]:
# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

# text libs
from sklearn.feature_extraction.text import TfidfVectorizer

import lime.lime_text
import lime.lime_tabular
import numpy as np
import pandas as pd
from tqdm import tqdm

from enum import Enum
import random
from scipy.sparse import vstack

In [ ]:
from folktables import ACSDataSource, ACSIncome

# local libraries
import exp_utils as eu

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
class Data(Enum):
    REVIEWS = 1
    INCOME = 2


Data = Enum("Data", ["REVIEWS", "INCOME"])

In [5]:
# set dataset
# dataset = Data.REVIEWS
DS = Data.INCOME
dataset_str = str(DS).split(".")[1]

In [8]:
if DS == Data.REVIEWS:
    train_df = pd.read_csv("data/reviews_train.csv")
    test_df = pd.read_csv("data/reviews_test.csv")

    tf_vec = TfidfVectorizer(lowercase=False, stop_words="english")
    X_train_vec = tf_vec.fit_transform(train_df["text"].values).toarray()
    X_test_vec = tf_vec.transform(test_df["text"].values).toarray()
    X_train = train_df["text"].values
    X_test = test_df["text"].values
    y_train = train_df["label"]
    y_test = test_df["label"]

elif DS == Data.INCOME:
    state = "HI"
    year = "2018"
    data_source = ACSDataSource(survey_year=year, horizon="1-Year", survey="person")
    acs_data = data_source.get_data(states=[state], download=True)
    features, label, group = ACSIncome.df_to_numpy(acs_data)
    X_train, X_test, y_train, y_test = train_test_split(
        features, label, test_size=0.20, random_state=42
    )
    scale = StandardScaler()
    X_train_vec = scale.fit_transform(X_train)
    X_test_vec = scale.transform(X_test)

In [9]:
# compare performance across different model classes
model_lin_svm = SVC(
    kernel="linear",
    probability=True,
)
model_lin_svm.fit(X_train_vec, y_train)
print(f"linear SVM: {model_lin_svm.score(X_test_vec, y_test)}")

linear SVM: 0.7472527472527473


In [29]:
if DS == Data.REVIEWS:
    n_feat = 10
elif DS == Data.INCOME:
    n_feat = 5

aug_results = []
random.seed(0)
for size in [800, 1600, 3200, len(y_train)]: 
    incl = np.asarray(random.sample(range(len(y_train)), size))
    ref_model = model_lin_svm
    for ind in tqdm(range(len(y_test))[:500]):
        test_instance = X_test_vec[ind].reshape(1, -1)
        pred = ref_model.predict(test_instance.reshape(1, -1))

        X_train_aug = vstack((X_train_vec[incl], test_instance))
        y_train_aug = np.vstack((y_train[incl].reshape(-1, 1), 1 - pred)).flatten()

        X_test_aug = vstack((X_test_vec[:ind], X_test_vec[ind + 1 :]))
        y_test_aug = np.delete(y_test, ind)

        aug_model = RandomForestClassifier()
        aug_model.fit(X_train_aug, y_train_aug)
        acc = aug_model.score(X_test_aug, y_test_aug.reshape(-1, 1))

        success = (aug_model.predict(test_instance) != pred)[0]

        if DS == Data.REVIEWS:
            explainer = lime.lime_text.LimeTextExplainer(class_names=["deceptive", "real"])
            c1 = make_pipeline(tf_vec, ref_model)
            c2 = make_pipeline(tf_vec, aug_model)
        else:
            explainer = lime.lime_tabular.LimeTabularExplainer(
                features,
                feature_names=ACSIncome._features,
                class_names=["<50k", "50k+"],
                discretize_continuous=False,
            )
            c1 = make_pipeline(scale, ref_model)
            c2 = make_pipeline(scale, aug_model)

        exp1 = explainer.explain_instance(
            X_test[ind], c1.predict_proba, num_features=n_feat
        )

        exp1_feat = np.asarray([feat for feat, weight in exp1.local_exp[1]])
        exp1_weights = np.asarray([weight for feat, weight in exp1.local_exp[1]])

        exp2 = explainer.explain_instance(
            X_test[ind], c2.predict_proba, num_features=n_feat
        )

        exp2_feat = np.asarray([feat for feat, weight in exp2.local_exp[1]])
        exp2_weights = np.asarray([weight for feat, weight in exp2.local_exp[1]])

        aug_results.append(
            {
                "ind": ind,
                "orig_pred": pred[0],
                "retrain_acc": acc,
                "top_feat": eu.top_features(exp1_feat, exp2_feat),
                "pos_pred": eu.pos_features_agg(
                    exp1_feat, exp1_weights, exp2_feat, exp2_weights
                ),
                "neg_pred": eu.neg_features_agg(
                    exp1_feat, exp1_weights, exp2_feat, exp2_weights
                ),
                "success": int(success),
            }
        )
        if ind % 10 == 0:
            model_aug_df = pd.DataFrame(aug_results)

            model_aug_df.to_csv(
                f"results/{dataset_str}_svm_aug_results_{len(y_train_aug)}.csv"
            )

100%|█████████████████████████████████████████████████████████████████████████████████████| 500/500 [17:34<00:00,  2.11s/it]


In [34]:
for n in [len(y_train)+1, 3201, 1601, 801,401]:
    model_aug_df = pd.read_csv(f"results/{dataset_str}_svm_aug_results_{n}.csv")
    print(n, model_aug_df.var()["success"], model_aug_df.std()["top_feat"])

6185 0.012894709610331923 0.14446879342593105
3201 0.007988872834321365 0.1571017066075198
1601 0.005025023188494429 0.1665631781773013
801 0.004065006858140407 0.0823609393322734
401 0.0013404813669488968 0.16043024415121315
